# Building first level models using _nipype_ and _SPM12_

## Base functionality for _megameta_ project

-------
#### History

* 4/16/19 cscholz - making changes to read in Banners and Image tasks of P1 as separate runs
* 4/9/19 cscholz - made small correction to make_contrast_list() (setting: -1/neg_length instead of -1/pos_length)
* 4/2/19 mbod - split out processing pipeline for revised workflow
* 3/28/19 mbod - update pipeline to include resampling to template & SPM path reference
* 3/23/19 mbod - include contrast definition in the config JSON file
* 3/9/19 mbod - updates from testing template with `darpa1`
* 2/27/19 mbod  - modify example notebook to make base functionality notebook

-----

### Description

* Set up a nipype workflow to use SPM12 to make first level models for _megameta_ task data (preprocessed using `batch8` SPM8 scripts) in BIDS derivative format   


-------------------

### Template variables

* Specify the following values:
    1. project name - should be name of folder under `/data00/project/megameta`, e.g. `project1`
    2. filename for JSON model specification (should be inside `model_specification` folder), e.g. `p1_image_pmod_likeme.json`
    3. TR value in seconds
 



-------------------

### Setup

* import required modules and define parameters

In [95]:
import os  # system functions

# NIYPE FUNCTIONS
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.modelgen as model   # model specification
from nipype.interfaces.base import Bunch
from nipype.algorithms.misc import Gunzip

from itertools import combinations

from nilearn import plotting, image
from nistats import thresholding

import glob
from IPython.display import Image


import scipy.io as sio
import numpy as np
import json
import pandas as pd

In [1]:
# NC commented out 04/19/19
#MODEL_SPEC_FILE = 'project1_model_BANNER_IMAGE_pmod_pop_rank.json'
#PROJECT_NAME = 'project1'
#TR=2.0

#### Matlab path


In [97]:
# Set the way matlab should be called
mlab.MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")
# If SPM is not in your MATLAB path you should add it here
mlab.MatlabCommand.set_default_paths(PATH_TO_SPM_FOLDER)

### Parameters

* These need to be reformatted to be consistent
* as data is not smoothed commented out the `fwhm_size` param - but data probably has a value

#### Load JSON model config

In [98]:
JSON_MODEL_FILE = os.path.join('/data00/projects/megameta/scripts/jupyter_megameta/first_level_models',
                               PROJECT_NAME, 'model_specifications',
                               MODEL_SPEC_FILE)

In [99]:
with open(JSON_MODEL_FILE) as fh:
    model_def = json.load(fh)

In [100]:
#cscholz: Using a task pattern instead of a simple name to read in 2 tasks
TASK_NAME = model_def['TaskName']
RUNS = model_def['Runs']
MODEL_NAME = model_def['ModelName']
PROJECT_NAME = model_def['ProjectID']

TASK_PATTERN = "[ib][am]*"   # hack for combined model look for task names that have first two characters "im" or "ba"

In [101]:
PROJECT_DIR = os.path.join('/data00/projects/megameta', PROJECT_NAME)
SUBJ_DIR = os.path.join(PROJECT_DIR, 'derivatives', 'nipype', 'resampled_and_smoothed')


task_func_template = "sr{PID}_task-{TASK}_run-0{RUN}_space-MNI152-T1-1mm_desc-preproc_bold.nii"


subject_list = [subj for subj in os.listdir(SUBJ_DIR) 
                   if os.path.exists(os.path.join(SUBJ_DIR,subj,'medium', 'fwhm_8',
                                        task_func_template.format(PID=subj, TASK=TASK_NAME[0], RUN=1)))
                   or os.path.exists(os.path.join(SUBJ_DIR,subj,'medium', 'fwhm_8',
                                        task_func_template.format(PID=subj, TASK=TASK_NAME[1], RUN=1)))]

output_dir = os.path.join(PROJECT_DIR,'derivatives', 'nipype','model_{}_{}_{}'.format(TASK_NAME[0].upper(),TASK_NAME[1].upper(), MODEL_NAME))        # name of 1st-level output folder
working_dir = os.path.join(PROJECT_DIR, 'working', 
                           'nipype', 'workingdir_model_{}_{}_{}'.format(TASK_NAME[0].upper(),TASK_NAME[1].upper(), MODEL_NAME))   # name of 1st-level working directory



In [ ]:
# check to see if output and work directories exist

if not os.path.exists(output_dir):
    os.makedirs(output_dir) 

if not os.path.exists(working_dir):
    os.makedirs(working_dir)

In [ ]:

try:
    subject_list = [ s for s in subject_list if s not in exclude_subjects ]
    print('\n\nApplied subject inclusion list:\n\t',' '.join(exclude_subjects))
except:
    print('\n\nNo subject exclusions applied')

try:
    subject_list = [ s for s in subject_list if s in include_subjects ]
    print('\n\nApplied subject inclusion list:\n\t',' '.join(include_subjects))
except:
    print('\n\nNo subject inclusions applied')

    
print('\n\nSUBJECT LIST IS:\n\t', ' '.join(subject_list))

### Utility functions for subject info and contrasts

### Setup design matrix data for subject

* need a function to set up the nipype `Bunch` format used
    * https://nipype.readthedocs.io/en/latest/users/model_specification.html
* read the onsets/dur/conditions from task logs and extract needed data

In [102]:
def get_subject_info(subject_id, model_path, DEBUG=False):
    '''
    1. load model specification from JSON spec file
    2. get confound file for subject for task to add to design matrix     
    3. get task spec CSV for subject for task
    4. setup subject info structure
    '''
    
    import os
    import pandas as pd
    import json
    
    from nipype.interfaces.base import Bunch
    
    
    def make_pmod(df, conditions, pmods={}, normalize='mean'):
        
        pmod = []
        
        for cond in conditions:
        
            
            if not pmods.get(cond):
                pmod.append(None)
            else:
                df2 = df[df.trial_type==cond]
                
                pmod_name = pmods.get(cond)
                
                #pmod = [pmod] if not type(pmods) is list else pmod
                
                # MAKE SURE THERE IS VARIANCE IN PMOD VECTOR
                if df2[pmod_name].var()==0:
                    #df2[pmod_name]+=0.001
                    pmod.append(None)
                    continue
                    
                    
                    
                # APPLY NORMALIZATION
                if normalize=='mean':
                    df2[pmod_name] = df2[pmod_name] - df2[pmod_name].mean()

                pmod.append(Bunch(name=[pmod_name],
                      param=[df2[pmod_name].values.tolist()
                            ],
                      poly=[1]
                     ))
        
        return pmod
    
    

    def map_spec_to_model(spec_df,model):
        """
        Maps spec trial names to model contrast trials.

        Args:
            spec: the events.tsv spec file
            model: the model.json file

        Returns:
            pandas dataframe object
        """

        spec=spec_df.copy()

        for con in model['Conditions']:
            spec_trials = model['Conditions'][con]
            spec.loc[spec.trial_type.isin(spec_trials),'trial_type'] = con
            spec.onset.sort_values()

        return spec
    
    
    with open(model_path) as fh:
        model_def = json.load(fh)
    
    
    pmod = None if not model_def.get('Modulators') else []
    
    TASK_NAME = model_def['TaskName']
    TASK_RUNS = model_def['Runs']
    MODEL_NAME = model_def['ModelName']
    PROJECT_ID = model_def['ProjectID']
    
    #cscholz hack to allow for missing conditions per run
    #condition_names = list(model_def['Conditions'].keys())
   
    
    PROJECT_DIR = os.path.join('/data00/projects/megameta', PROJECT_ID)
    SUBJ_DIR = os.path.join(PROJECT_DIR,'derivatives', 'batch8')
    
    realign_files = []
    subject_info = []
    

    
    # check to see which runs exist for subject
    # by looking for appropriate events.tsv files
    # this could (should?) also include looking for the nifti file?
    ## CSCHOLZ: putting new runs_for_subj function to allow for multiple tasks
    #runs_for_subj = [run for run in TASK_RUNS
    #                if 
    #                os.path.exists(os.path.join(SUBJ_DIR, subject_id, 'func',
    #                                '{}_task-{}_run-0{}_events.tsv'.format(subject_id, 
    #                                                                       TASK_NAME,
    #                                                                       run)))
    #                ]
    runs_for_subj = {}
    for task in TASK_RUNS:
        cur_run_list= [run for run in TASK_RUNS[task]
                       if 
                       os.path.exists(os.path.join(SUBJ_DIR, subject_id, 'func',
                                    '{}_task-{}_run-0{}_events.tsv'.format(subject_id, 
                                                                           task,
                                                                           run)))
                      ]
        runs_for_subj[task]=cur_run_list
    
    if DEBUG:
        print("runs_for_subj", runs_for_subj)
        print("checked paths:")
        for run in TASK_RUNS:
            print('\t', os.path.join(SUBJ_DIR, subject_id, 'func',
                                    '{}_task-{}_run-0{}_events.tsv'.format(subject_id, 
                                                                           TASK_NAME,
                                                                           run)))
        print("TASK NAME", TASK_NAME)
        print("pmod", pmod)
        print("TASK_RUNS", TASK_RUNS)
        print("subject_id", subject_id)
    
    # Cscholz looping through tasks and then through run per task to allow for multiple tasks
    for task in runs_for_subj:
        
        for run_num, _ in enumerate(runs_for_subj[task],1):
    
            #cscholz adding hack to allow for missing conditions per run
            condition_names=list(model_def['Conditions'].keys())
            
            events_df = pd.read_csv(os.path.join(SUBJ_DIR, subject_id, 'func',
                                             '{}_task-{}_run-0{}_events.tsv'.format(subject_id, 
                                                                                    task,
                                                                                    run_num)),
                               sep='\t')

            #cscholz removing conditions not present this run
            remove_conds=[]
            for cond in condition_names:
                present_conds=0
                for orig_cond in model_def['Conditions'][cond]:
                    if orig_cond in list(events_df['trial_type']):
                        present_conds=present_conds+1
                if present_conds==0:
                    remove_conds.append(cond)
            
            condition_names=[x for x in condition_names if x not in remove_conds]
            
            onsets_df = map_spec_to_model(events_df, model_def)
        
           #editing filenames to allow for multiple tasks     
            realign_file = os.path.join(PROJECT_DIR, 'working','nipype',
                                        'workingdir_model_{}_{}_{}'.format(TASK_NAME[0].upper(),TASK_NAME[1].upper(),MODEL_NAME),
                                        '{}-task-{}-run-0{}-realign.txt'.format(subject_id, task, run_num))

            confound_file=os.path.join(SUBJ_DIR, subject_id, 'func',
                                   '{}_task-{}_run-0{}_desc-confounds-regressors.tsv'.format(subject_id, 
                                                                                             task,
                                                                                             run_num)
                                   )

            confound_df = pd.read_csv(confound_file, sep='\t')

            cols_to_use = [ 'TransX','TransY', 'TransZ', 'RotX', 'RotY', 'RotZ']

            confound_df[cols_to_use].to_csv(realign_file, 
                                        header=False, 
                                        index=False,
                                        sep='\t')

            realign_files.append(realign_file)

            onsets = []
            dur = []
            #cscholz hack to allow for missing conditions
            for cond in condition_names:
            #for cond in model_def['Conditions']:
                onsets.append(onsets_df[onsets_df.trial_type==cond].onset.values)
                dur.append(onsets_df[onsets_df.trial_type==cond].duration.values)

            
         
            #pmod = make_pmod(rdf, condition_names)   
        
            if model_def.get('Modulators'):
                pmod = make_pmod(onsets_df, condition_names, 
                                pmods=model_def['Modulators'])     

 
        
        
            subject_info.append(Bunch(conditions=condition_names,
                         onsets=onsets,
                         durations=dur,
                         amplitudes=None,
                         tmod=None,
                         pmod=pmod,
                         regressor_names=None,
                         regressors=None))
    
    #cscholz: hack to allow for missing conditions
    condition_names=model_def['Conditions'].keys()
    
    DM_regressors = []
    for cond in condition_names:
        DM_regressors.append(cond)
        if pmod and model_def['Modulators'].get(cond):
            DM_regressors.append('{}x{}^1'.format(cond, model_def['Modulators'].get(cond)))
    
    
    return subject_info, realign_files, DM_regressors

In [103]:
get_subject_info('sub-P100',os.path.join('/data00/projects/megameta/scripts/jupyter_megameta/first_level_models',
                               PROJECT_NAME, 'model_specifications',
                               MODEL_SPEC_FILE))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


([Bunch(amplitudes=None,
        conditions=['message'],
        durations=[array([ 14.610247,  14.612541,  14.229069,  15.062439,  20.728531,
                 16.729065,  13.94577 ,  16.062469,  15.895508,  16.545639,
                 17.545517,  21.01178 ,  14.395844,  30.010376,  19.512085,
                 13.895721,  15.0625  ,  20.678406,  23.327942,  18.545319,
                 14.328979,  18.94519 ,  20.44519 ])],
        onsets=[array([   0.      ,   22.411564,   44.25906 ,   66.189499,   90.51973 ,
                 118.7995  ,  143.163452,  164.310532,  190.024353,  213.037949,
                 237.968185,  265.698029,  296.427643,  318.108521,  355.286804,
                 383.033783,  405.264099,  428.744568,  457.124542,  489.720367,
                 517.0672  ,  538.481079,  565.510925])],
        pmod=[Bunch(name=['population_CTR_rank'],
               param=[[-6.0,
                 -1.0,
                 11.0,
                 -11.0,
                 6.0,
              

### Set up contrasts

* This part of the template needs work to provide a cleaner way to specify contrasts
* Could use the same vector contrasts approach as we have in batch8 and then have a function to convert this into the list of list data structure that nipype spm contrasts node looks for

In [ ]:
def make_contrast_list(subject_id, condition_names, model_path, DEBUG=False):
    
    import json
    
    condition_names.append('constant')

    cont = []
    for idx, cname in enumerate(condition_names):
        ccode = [0 if pos!=idx else 1 for pos in range(len(condition_names))]
        cont.append([cname, 'T', condition_names, ccode])

    # add custom contrasts from the JSON model file
    with open(model_path) as fh:
        model_def = json.load(fh)
    
    contrasts = model_def.get('Contrasts')
    
    if not contrasts:
        return cont
    
    for contrast in contrasts:
            
        cname = contrast['name']
        
        
        
        pos_idx = [condition_names.index(p) for p in contrast['pos']]
        neg_idx = [condition_names.index(n) for n in contrast['neg']]
        
        pos_length = len(contrast['pos'])
        neg_length = len(contrast['neg'])
        
        ccode = []
        for idx, _ in enumerate(condition_names):
            if idx in pos_idx:
                ccode.append(1/pos_length)
            elif idx in neg_idx:
                ccode.append(-1/neg_length)
            else:
                ccode.append(0)

        cont.append([cname, 'T', condition_names, ccode])
        
                
        if DEBUG:
            print(contrast)
            print(ccode)
            
    return cont

## Set up processing nodes for modeling workflow

#### Specify model node

In [ ]:
# SpecifyModel - Generates SPM-specific Model
modelspec = pe.Node(model.SpecifySPMModel(concatenate_runs=False,
                                 input_units='secs',
                                 output_units='secs',
                                 time_repetition=TR,
                                 high_pass_filter_cutoff=128),
                                 output_units = 'scans',
                 name="modelspec")

#### Level 1 Design node

** TODO -- get the right matching template file for fmriprep **

* ??do we need a different mask than:

    `'/data00/tools/spm8/apriori/brainmask_th25.nii'`

In [ ]:
# Level1Design - Generates an SPM design matrix
level1design = pe.Node(spm.Level1Design(bases={'hrf': {'derivs': [0, 0]}},
                                 timing_units='secs',
                                 interscan_interval=TR,
                                 model_serial_correlations='none', #'AR(1)',
                                 mask_image = '/data00/tools/spm8/apriori/brainmask_th25.nii',
                                 global_intensity_normalization='none'
                                       ),
                    name="level1design")

#### Estimate Model node

In [ ]:
# EstimateModel - estimate the parameters of the model
level1estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1}),
                      name="level1estimate")

#### Estimate Contrasts node

In [ ]:
# EstimateContrast - estimates contrasts
conestimate = pe.Node(spm.EstimateContrast(), name="conestimate")

## Setup pipeline workflow for level 1 model

In [ ]:
# Initiation of the 1st-level analysis workflow
l1analysis = pe.Workflow(name='l1analysis')

# Connect up the 1st-level analysis components
l1analysis.connect([(modelspec, level1design, [('session_info',
                                                'session_info')]),
                    (level1design, level1estimate, [('spm_mat_file',
                                                     'spm_mat_file')]),
                    (level1estimate, conestimate, [('spm_mat_file',
                                                    'spm_mat_file'),
                                                   ('beta_images',
                                                    'beta_images'),
                                                   ('residual_image',
                                                    'residual_image')])
                   
                    ])

## Set up nodes for file handling and subject selection

### `getsubjectinfo` node 

* Use `get_subject_info()` function to generate spec data structure for first level model design matrix

In [ ]:
# Get Subject Info - get subject specific condition information
getsubjectinfo = pe.Node(util.Function(input_names=['subject_id', 'model_path'],
                               output_names=['subject_info', 'realign_params', 'condition_names'],
                               function=get_subject_info),
                      name='getsubjectinfo')

In [ ]:
makecontrasts = pe.Node(util.Function(input_names=['subject_id', 'condition_names', 'model_path'],
                                     output_names=['contrasts'],
                                      function=make_contrast_list),
                    name='makecontrasts')

### `infosource` node

* iterate over list of subject ids and generate subject ids and produce list of contrasts for subsequent nodes

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = pe.Node(util.IdentityInterface(fields=['subject_id', 'model_path', 'resolution', 'smoothing']
                                   ),
                  name="infosource")

try:
    fwhm_list = smoothing_list
except:
    fwhm_list = [4,6,8]
    
try:
    resolution_list = resolutions
except:
    resolution_list = ['low','medium','high']

infosource.iterables = [('subject_id', subject_list),
                        ('model_path', [JSON_MODEL_FILE]*len(subject_list)),
                        ('resolution', resolution_list),
                        ('smoothing', ['fwhm_{}'.format(s) for s in fwhm_list])
                       ]



### `selectfiles` node

* match template to find source files (functional) for use in subsequent parts of pipeline

In [ ]:
# SelectFiles - to grab the data (alternativ to DataGrabber)

## TODO: here need to figure out how to incorporate the run number and task name in call
templates = {'func': '{subject_id}/{resolution}/{smoothing}/sr{subject_id}_task-'+TASK_PATTERN+'_run-0*_space-MNI152-T1-1mm_desc-preproc_bold.nii'}          

selectfiles = pe.Node(nio.SelectFiles(templates,
                               base_directory='/data00/projects/megameta/{}/derivatives/nipype/resampled_and_smoothed'.format(PROJECT_NAME)),
                      working_dir=working_dir,
                   name="selectfiles")


### Specify datasink node

* copy files to keep from various working folders to output folder for model for subject

In [ ]:
# Datasink - creates output folder for important outputs
datasink = pe.Node(nio.DataSink(base_directory=SUBJ_DIR,
                         parameterization=True, 
                         #container=output_dir      
                               ),
                name="datasink")

datasink.inputs.base_directory = output_dir


# Use the following DataSink output substitutions
substitutions = []
subjFolders = [('_model_path.*resolution_(low|medium|high)_smoothing_(fwhm_\\d{1,2})_subject_id_sub-.*/(.*)$', '\\1/\\2/\\3')]
substitutions.extend(subjFolders)
datasink.inputs.regexp_substitutions = substitutions



---------

## Set up workflow for whole process

In [ ]:
pipeline = pe.Workflow(name='first_level_model_{}_{}_{}'.format(TASK_NAME[0].upper(),
                                                                TASK_NAME[1].upper(),
                                                                MODEL_NAME))
pipeline.base_dir = os.path.join(SUBJ_DIR, working_dir)


pipeline.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                             ('resolution', 'resolution'),
                                             ('smoothing', 'smoothing')
                                            ]),
                  (infosource, getsubjectinfo, [('subject_id', 'subject_id'),
                                                ('model_path', 'model_path')
                                               ]),
                  (infosource, makecontrasts, [('subject_id', 'subject_id'),
                                               ('model_path', 'model_path')
                                              ]),
                  (getsubjectinfo, makecontrasts, [('condition_names', 'condition_names')]),
                  
                 (getsubjectinfo, l1analysis, [('subject_info',
                                                 'modelspec.subject_info'),
                                                ('realign_params',
                                                   'modelspec.realignment_parameters')]),
                  (makecontrasts, l1analysis, [('contrasts',
                                             'conestimate.contrasts')]),
                  
                  
                  (selectfiles, l1analysis, [('func',
                                          'modelspec.functional_runs')]),
                  
                                    
                  (infosource, datasink, [('subject_id','container')]),
                  (l1analysis, datasink, [('conestimate.spm_mat_file','@spm'),
                                          ('level1estimate.beta_images','@betas'),
                                          ('level1estimate.mask_image','@mask'),
                                          ('conestimate.spmT_images','@spmT'),
                                          ('conestimate.con_images','@con'),
                                          ('conestimate.spmF_images','@spmF')
                                         ])
                 ]
)
                  